In [80]:
import pandas as pd

In [81]:
energy = pd.read_csv("energydata_complete.csv")

In [82]:
energy.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


# Observations after Exploratory data analysis on energy dataset:
1. We have appliance energy consumption time series dataset.
2. Also, the dataset is numeric, as we don't have any categorical features in it.
3. The data type for date feature needs to be datetime.
4. The date scope of the records is from 11th January 2016 to 27th May 2016 with 10 min interval in each record.
5. Total Number of records are 19735.
6. We do not have any missing vaules in energy dataset.
7. We have rv1 and rv2 features with perfect correlation and found to be same. Hence we can remove one of the feature from the dataset.
8. 90.29% of data for the appliance energy consumption is between 0-200 Wh.
9. 77.28% of data have lights off in the house, i.e. no energy consumption were recorded for the lights.
10. We have different temperature and humidity features. Also, features like windspeed, visibility and pressure has been recorded.
11. All temperature features are significantly correlated to each other.
12. All humidity featured has good correlation with each other except for RH_5, RH_6 and RH_out, due to the surrounding were it measured.
13. Feature T9 shows high correlation with T3, T4, T5, T7 and feature T5 has high correlation with T9, T3, T1. Hence can be consider as a redundant feature, giving us scope to eliminate those features.
14. Also, T6 and T_out are exterior temperatue features, and shows high degree of correlations. Thus we can get rid of one of these featurs.
15. To get more insight on the time series data for energy consumption we need to some more derived feature.
16. Features like month, time, DOY (Day of year), Only_Date and Date of week has been derived from date feature and added to the main energy dataset.
17. From the month January to February the appliance energy consumption increases and then decreases in the month of March.
18. Again the rise in the appliance energy increase in April and further drop in the month of May.
19. Similarly, there was a rise in the light usage for the month February and then gradual decrease in March, April and May.
20. (Average) Appliance energy utilizations is low when lights are OFF.
21. (Average) Appliance energy utilizations is high when lights are ON.
22. Also, Appliance energy utilizations follows a trand with respect to time.
23. For the night time (00:00:00 to 06:00:00) the energy consumption is lowest.
24. From the time 06:00:00 energy consumption started rising, and got steady till time 17:20:00
25. Then the peak hours started till 19:00:00
26. After that, the energy utilization started decreasing.
27. for normal hours 07:00:00 to 17:00:00 the energy utilization on weekend increases as compared to weekdays.
28. But for the peak hours ie. 17:00:00 to 19:00:00 the energy utilization decreases.
29. Feature rv1 does not have any specific pattern in the Appliances energy consumption over time.
30. Thus, giving us scope to think on eliminating the rv1.


In [83]:
def featureengineering(energy):
    # Converting datatype of Date column to date time
    energy['date'] = pd.to_datetime(energy['date'])

    # Removing rv2 feature
    del energy['rv2']
    
    # Removing T9 feature
    del energy['T9']
    
    # Removing T6 feature
    del energy['T6']
    
    # Removing rv1
    del energy['rv1']
    
    # To get the month for that record
    energy['month'] = energy['date'].dt.month

    # To get the specific time for that record
    #energy['time'] = energy['date'].dt.time

    #---------------------------------------------------
    p = []
    q = []
    for i in energy['date']:
        p.append(i.strftime("%j"))
        q.append(i.hour * 60 + i.minute)
        
    p=list(map(int, p))
    #---------------------------------------------------

    # To get the Day of the year for that record
    energy['DOY'] = pd.DataFrame({'DOY': p})

    # To get the minutes from midnight for that record
    energy['NSM'] = pd.DataFrame({'NSM': q})

    # To get the only date 
    #energy['Only_Date'] = energy['date'].dt.date
    #energy['Only_Date'] = pd.to_datetime(energy['Only_Date'])
    
    # To get the weekday for that record
    energy['Day of Week'] = energy['date'].dt.weekday
    
    # Now as we have DOY (Day of year) and MNM, we can remove date variable.
    del energy['date']
    
    # Generating training and testing dataset
    from sklearn.model_selection import train_test_split
    
    energy_train, energy_test = train_test_split(energy, test_size=0.2)

    #energy.sort_index()
    #energy_train = energy[0:int(energy.shape[0]*0.8)]
    #energy_test = energy[int(energy.shape[0]*0.8):]
    
    return energy_train, energy_test

In [84]:
energy.columns

Index(['date', 'Appliances', 'lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3',
       'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8',
       'RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed',
       'Visibility', 'Tdewpoint', 'rv1', 'rv2'],
      dtype='object')

In [85]:
energy_train, energy_test = featureengineering(energy)

In [86]:
energy_train.columns

Index(['Appliances', 'lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4',
       'RH_4', 'T5', 'RH_5', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'RH_9',
       'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility',
       'Tdewpoint', 'month', 'DOY', 'NSM', 'Day of Week'],
      dtype='object')

In [87]:
energy_train.head()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,month,DOY,NSM,Day of Week
7494,90,0,21.600000,38.07,20.000000,38.137143,20.70,37.95,20.861429,34.652857,...,6.100000,749.600000,72.000000,2.000000,40.0,1.300000,3,63,1080,3
977,40,0,19.600000,41.23,18.290000,41.590000,19.79,40.70,19.066667,37.966667,...,-1.533333,758.483333,75.000000,3.000000,40.0,-5.466667,1,18,710,0
6774,90,10,20.890000,34.89,19.500000,33.466667,20.79,35.09,20.100000,32.090000,...,4.800000,751.300000,58.000000,5.000000,29.0,-2.800000,2,58,1080,5
2480,20,0,18.890000,43.79,18.000000,44.126667,19.70,43.50,17.600000,43.966667,...,2.833333,765.700000,88.666667,4.333333,32.0,1.133333,1,28,1340,3
11273,70,20,21.926667,40.56,19.666667,42.466667,22.89,38.90,21.566667,40.200000,...,9.766667,750.016667,79.333333,7.666667,40.0,6.266667,3,89,1430,1


In [88]:
energy_test.columns

Index(['Appliances', 'lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4',
       'RH_4', 'T5', 'RH_5', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'RH_9',
       'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility',
       'Tdewpoint', 'month', 'DOY', 'NSM', 'Day of Week'],
      dtype='object')

In [89]:
energy.columns

Index(['Appliances', 'lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4',
       'RH_4', 'T5', 'RH_5', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'RH_9',
       'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility',
       'Tdewpoint', 'month', 'DOY', 'NSM', 'Day of Week'],
      dtype='object')

In [90]:
energy.shape

(19735, 28)

In [91]:
energy_test.shape

(3947, 28)

In [92]:
energy_train.shape

(15788, 28)

In [93]:
energy.dtypes

Appliances       int64
lights           int64
T1             float64
RH_1           float64
T2             float64
RH_2           float64
T3             float64
RH_3           float64
T4             float64
RH_4           float64
T5             float64
RH_5           float64
RH_6           float64
T7             float64
RH_7           float64
T8             float64
RH_8           float64
RH_9           float64
T_out          float64
Press_mm_hg    float64
RH_out         float64
Windspeed      float64
Visibility     float64
Tdewpoint      float64
month            int64
DOY              int64
NSM              int64
Day of Week      int64
dtype: object

In [94]:
energy.head()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,month,DOY,NSM,Day of Week
0,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,...,6.600000,733.5,92.0,7.000000,63.000000,5.3,1,11,1020,0
1,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,...,6.483333,733.6,92.0,6.666667,59.166667,5.2,1,11,1030,0
2,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,...,6.366667,733.7,92.0,6.333333,55.333333,5.1,1,11,1040,0
3,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,...,6.250000,733.8,92.0,6.000000,51.500000,5.0,1,11,1050,0
4,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,...,6.133333,733.9,92.0,5.666667,47.666667,4.9,1,11,1060,0
